# Zero-Shot Evaluation of GeneFormer

Following https://huggingface.co/ctheodoris/Geneformer/tree/main

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [18]:
from transformers import AutoConfig, AutoModelForMaskedLM
import torch

# Define the paths to the config and model files
# config_path = '/home/icb/till.richter/git/Geneformer/fine_tuned_models/geneformer-6L-30M_CellClassifier_cardiomyopathies_220224/config.json'
# model_path = '/home/icb/till.richter/git/Geneformer/fine_tuned_models/geneformer-6L-30M_CellClassifier_cardiomyopathies_220224/pytorch_model.bin'

config_path = '/home/icb/till.richter/git/Geneformer/geneformer-12L-30M/config.json'
model_path = '/home/icb/till.richter/git/Geneformer/geneformer-12L-30M/pytorch_model.bin'

# Load the configuration and model using the local paths
config = AutoConfig.from_pretrained(config_path)
model = AutoModelForMaskedLM.from_pretrained(model_path, config=config)


In [28]:
from self_supervision.estimator.cellnet import EstimatorAutoEncoder

DATA_PATH = '/lustre/groups/ml01/workspace/till.richter/merlin_cxg_2023_05_15_sf-log1p'  # this is a shared directory in which the data is stored
estim = EstimatorAutoEncoder(DATA_PATH, hvg=False)
estim.init_datamodule(batch_size=512) 

In [29]:
import requests
from tqdm.notebook import tqdm  # for progress bar in Jupyter

# Replace 'YOUR_API_TOKEN' with your actual API token
API_TOKEN = "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"
API_URL = "https://api-inference.huggingface.co/models/ctheodoris/Geneformer"
headers = {"Authorization": f"Bearer {API_TOKEN}"}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

# Assuming `estim` is defined and has the method `datamodule.test_dataloader()`
test_dataloader = estim.datamodule.test_dataloader()
y_pred = []

# Iterate over your test dataset
for batch in tqdm(test_dataloader, desc="Evaluating"):
    # Assuming batch is a dictionary with an 'inputs' key that contains the data in the correct format
    inputs = batch["inputs"]
    try:
        # Query the model and get prediction
        output = query({"inputs": inputs})
        # Process the output to extract the prediction part you are interested in
        # This will depend on the output format from GeneFormer
        prediction = output['prediction']  # or however the output is structured
        
        y_pred.append(prediction)
    except Exception as e:
        print(f"An error occurred: {e}")
        y_pred.append(None)  # Append None or some error indicator as needed

# Now `y_pred` contains all your predictions.


Evaluating:   0%|          | 0/6736 [00:00<?, ?it/s]

TypeError: tuple indices must be integers or slices, not str

In [19]:

tokenizer = AutoTokenizer.from_pretrained('/home/icb/till.richter/git/Geneformer/geneformer-12L-30M/')


OSError: Can't load tokenizer for '/home/icb/till.richter/git/Geneformer/geneformer-12L-30M/'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/home/icb/till.richter/git/Geneformer/geneformer-12L-30M/' is the correct path to a directory containing all relevant files for a BertTokenizerFast tokenizer.

In [21]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("fill-mask", model="/home/icb/till.richter/git/Geneformer/")

OSError: Can't load tokenizer for '/home/icb/till.richter/git/Geneformer/'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure '/home/icb/till.richter/git/Geneformer/' is the correct path to a directory containing all relevant files for a BertTokenizerFast tokenizer.

In [25]:
from geneformer import TranscriptomeTokenizer
tk = TranscriptomeTokenizer({"cell_type": "cell_type"}, nproc=16)
tk.tokenize_data("/lustre/groups/ml01/workspace/till.richter/GeneFormer/", 
                 "/lustre/groups/ml01/workspace/till.richter/GeneFormer/output/", 
                 "output_prefix_tokenization", 
                 file_format="h5ad")

Tokenizing /lustre/groups/ml01/workspace/till.richter/GeneFormer/log1p_cellxgene_val_adata.h5ad


OSError: Unable to open file (truncated file: eof = 89865408512, sblock->base_addr = 0, stored_eof = 270833284624)

In [ ]:
import requests
from tqdm.notebook import tqdm  # for progress bar in Jupyter

# Replace 'YOUR_API_TOKEN' with your actual API token
API_TOKEN = "YOUR_API_TOKEN"
API_URL = "https://api-inference.huggingface.co/models/ctheodoris/Geneformer"
headers = {"Authorization": f"Bearer {API_TOKEN}"}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

# Assuming `estim` is defined and has the method `datamodule.test_dataloader()`
test_dataloader = estim.datamodule.test_dataloader()
y_pred = []

# Iterate over your test dataset
for batch in tqdm(test_dataloader, desc="Evaluating"):
    # Assuming batch is a dictionary with an 'inputs' key that contains the data in the correct format
    inputs = batch["inputs"]
    try:
        # Query the model and get prediction
        output = query({"inputs": inputs})
        # Process the output to extract the prediction part you are interested in
        # This will depend on the output format from GeneFormer
        prediction = output['prediction']  # or however the output is structured
        
        y_pred.append(prediction)
    except Exception as e:
        print(f"An error occurred: {e}")
        y_pred.append(None)  # Append None or some error indicator as needed
